In [1]:
import pandas as pd
import numpy as np

![image](https://user-images.githubusercontent.com/872296/40033839-283a0aa2-57c8-11e8-98fd-399977bb49f0.png)

### Import the Data

pandas [`read_html`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html) method parses an HTML website and tries to pull tables out of it.

This website has two tables, one for each conference.

In [2]:
page = pd.read_html('https://www.mlssoccer.com/standings/mls/2017/')

In [4]:
t1 = page[0]

In [5]:
t1.head()

,Unnamed: 0,Unnamed: 1,Points,Unnamed: 3,Overall,Unnamed: 5,Home,Unnamed: 7,Away,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,#,Club,PTS,PPG,NaN,GP,W,L,T,GF,GA,GD,NaN,W-L-T,NaN,W-L-T
1,1,TORToronto FC,69,2.03,NaN,34,20,5,9,74,37,37,NaN,13-1-3,NaN,7-4-6
2,2,NYCNew York City FC,57,1.68,NaN,34,16,9,9,56,43,13,NaN,10-2-5,NaN,6-7-4
3,3,CHIChicago Fire,55,1.62,NaN,34,16,11,7,61,47,14,NaN,12-2-3,NaN,4-9-4
4,4,ATLAtlanta United FC,55,1.62,NaN,34,15,9,10,70,40,30,NaN,11-3-3,NaN,4-6-7


It made its best, but got a few columns wrong. For example, `'Overall'` is just a super column that groups "Games Played" (`GP`), "Won games" (`W`), etc. Here are all the columns:

In [6]:
t1.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Points', 'Unnamed: 3', 'Overall',
       'Unnamed: 5', 'Home', 'Unnamed: 7', 'Away', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15'],
      dtype='object')

We then drop some of those columns:

In [7]:
t1.drop(columns=['Overall', 'Unnamed: 12', 'Unnamed: 14'], inplace=True)

The real column names were actually parsed as the first row of the table:

In [8]:
t1.head(2)

,Unnamed: 0,Unnamed: 1,Points,Unnamed: 3,Unnamed: 5,Home,Unnamed: 7,Away,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 13,Unnamed: 15
0,#,Club,PTS,PPG,GP,W,L,T,GF,GA,GD,W-L-T,W-L-T
1,1,TORToronto FC,69,2.03,34,20,5,9,74,37,37,13-1-3,7-4-6


And these are the values:

In [9]:
t1.loc[0].values

array(['#', 'Club', 'PTS', 'PPG', 'GP', 'W', 'L', 'T', 'GF', 'GA', 'GD',
       'W-L-T', 'W-L-T'], dtype=object)

So we can just replace the column names with those of the first row:

In [10]:
t1.columns = t1.loc[0].values

In [11]:
t1.head(2)

,#,Club,PTS,PPG,GP,W,L,T,GF,GA,GD,W-L-T,W-L-T
0,#,Club,PTS,PPG,GP,W,L,T,GF,GA,GD,W-L-T,W-L-T
1,1,TORToronto FC,69,2.03,34,20,5,9,74,37,37,13-1-3,7-4-6


We can now drop the first column:

In [12]:
t1.drop(index=0, inplace=True)
t1

,#,Club,PTS,PPG,GP,W,L,T,GF,GA,GD,W-L-T,W-L-T
1,1,TORToronto FC,69,2.03,34,20,5,9,74,37,37,13-1-3,7-4-6
2,2,NYCNew York City FC,57,1.68,34,16,9,9,56,43,13,10-2-5,6-7-4
3,3,CHIChicago Fire,55,1.62,34,16,11,7,61,47,14,12-2-3,4-9-4
4,4,ATLAtlanta United FC,55,1.62,34,15,9,10,70,40,30,11-3-3,4-6-7
5,5,CLBColumbus Crew SC,54,1.59,34,16,12,6,53,49,4,12-3-2,4-9-4
6,6,RBNYNew York Red Bulls,50,1.47,34,14,12,8,53,47,6,9-2-6,5-10-2
7,7,NENew England Revolution,45,1.32,34,13,15,6,53,61,-8,12-2-3,1-13-3
8,8,PHIPhiladelphia Union,42,1.24,34,11,14,9,50,47,3,10-4-3,1-10-6
9,9,MTLMontreal Impact,39,1.15,34,11,17,6,52,58,-6,8-8-1,3-9-5
10,10,ORLOrlando City SC,39,1.15,34,10,15,9,39,58,-19,7-5-5,3-10-4


### Repeating the process

For the second table, we can just create a function and do it all at once:

In [ ]:
def parse_mls_standing_table(table):
    table.drop(columns=['Overall', 'Unnamed: 12', 'Unnamed: 14'], inplace=True)
    table.columns = table.loc[0].values
    table.drop(index=0, inplace=True)
    return table

In [ ]:
t2 = parse_mls_standing_table(page[1].copy())

In [ ]:
t2.head()

### Merging both tables


Now we need to merge both tables to process them together. But before doing so, we need a way to remember each team's conference. I'll go with the easiest approach, just a 'Conference' column. A multi-index could also have been an option.

In [ ]:
t1['Conference'] = 'Eastern'
t2['Conference'] = 'Western'

In [ ]:
t2

Finally, putting altogether:

In [ ]:
standings = pd.concat([t1, t2])
standings